# Do it 자연어 처리 3장 숫자 세계로 떠난 자연아
## 3.1 미리 학습된 언어 모델
### 언어 모델
- 단어 시퀀스에 확률을 부여하는 모델
- P(w1,..,wn) = 1에서 n까지 다 곱하기 (P(wi|w1,..,wn)) or P(w|context)
- -> 이전 단어들이 주어졌을 때 다음 단어가 나타날 확률을 부여하는 모델

### 순방향 언어 모델
- 사람이 이해하는 순서대로 계산하는 모델(GPT, ELMo)

### 역방향 언어 모델
- 순방향에서 방향만 바뀔뿐 전체 단어 시퀀스가 나타날 확률을 계산(ELMo)

### 마스크 언어 모델
- 학습 대상 문장에 빈칸을 만들고 해당 빈칸에 적절한 단어가 무엇일지 분류(BERT)
- 순방향, 역방향 언어 모델과 달리 문장의 전체 맥락을 참고할 수 있다.
- 양방향성 -> 맞힐 단어 앞뒤를 모두 본다.

### 스킵-그램 모델
- 어떤 단어 앞뒤에 특정 범위를 정해두고 이 범위 내에 어떤 단어가 올지 분류하는 과정
- 컨텍스트로 설정한 단어 주변에 어떤 단어들이 분포해 있는지 학습(Word2Vec)

---
## 3.2 트랜스포머 살펴보기
### 시퀀스-투-시퀀스
- 특정 속성을 지닌 시퀀스를 다른 속성의 시퀀스로 변환하는 작업 (ex. 번역 등)

### 인코더와 디코더
- 인코딩: 인코더가 소스 시퀀스 정보를 압축
- 디코딩: 디코더가 타깃 시퀀스를 생성하는 과정

### 트랜스포머
- 인코더와 디코더 입력이 주어졌을 때 정답에 해당하는 단어의 확률값을 높이는 방식으로 학습

### 셀프 어텐션
- 어텐션: 중요한 요소에 더 집중해 성능을 끌어 올리는 기법
- cnn: 필터 크기를 넘어서는 문맥은 읽어내기 어려움
- rnn: 오래전 읽었던 단어는 잊어버림
- 셀프어텐션: 문맥 전체를 고려하고 1:1로 단어가 바라보기에 길이가 길어도 단어를 잊어버리지 않음
- 쿼리, 키, 밸류가 서로 영향을 주고 받으며 문장의 의미를 계산
- 트랜스포머는 셀프 어텐션을 블록(레이어) 수 만큼 반복

---
## 3.3 셀프 어텐션 동작 원리


In [1]:
# .1 변수 정의
import torch

x = torch.tensor([
    [1.0, 0.0, 1.0, 0.0],
    [0.0, 2.0, 0.0, 2.0],
    [1.0, 1.0, 1.0, 1.0],
])
w_query = torch.tensor([
    [1.0, 0.0, 1.0],
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0]
])
w_key = torch.tensor([
    [0.0, 0.0, 1.0],
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 0.0],
    [1.0, 1.0, 0.0]
])
w_value = torch.tensor([
    [0.0, 2.0, 0.0],
    [0.0, 3.0, 0.0],
    [1.0, 0.0, 3.0],
    [1.0, 1.0, 0.0]
])

In [2]:
# .2 쿼리, 키, 밸류 만들기
# matmul: 행렬곱
keys = torch.matmul(x, w_key)
querys = torch.matmul(x, w_query)
values = torch.matmul(x, w_value)

In [3]:
# .3 어텐션 스코어 만들기
attn_scores = torch.matmul(querys, keys.T)
attn_scores

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

In [4]:
# .4 소프트맥스 확률값 만들기
import numpy as np
from torch.nn.functional import softmax
key_dim_sqrt = np.sqrt(keys.shape[-1])
attn_probs = softmax(attn_scores / key_dim_sqrt, dim=-1)
attn_probs

tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])

In [5]:
# .5 소프트맥스 롹률과 밸류를 가중합하기
weighted_values = torch.matmul(attn_probs, values)
weighted_values

tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])

### 멀티 헤드 어텐션
- 셀프 어텐션을 동시에 여러 번 수행하는 것
- 개별 헤드의 셀프 어텐션 수행결과를 이어 붙인 행렬에 W0행렬 곱을 곱함
- W0: 셀프 어텐션 수행 결과 행렬의 열 수 * 목표 차원 수
- 최종 수행 결괴: 입력 단어 수 * 목표 차원 수